In [1]:
%load_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

sns.set(style='ticks', context='talk', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline


import flotilla
flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)
not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)

psi = study.splicing.singles.ix[not_outliers]
grouped = psi.groupby(study.sample_id_to_phenotype)
psi_filtered = grouped.apply(lambda x: x.dropna(axis=1, thresh=20))

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-05-13 14:31:58	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure2_modalities_bayesian/datapackage.json
2016-05-13 14:31:58	Parsing datapackage to create a Study object
2016-05-13 14:32:35	Initializing Study
2016-05-13 14:32:35	Initializing Predictor configuration manager for Study
2016-05-13 14:32:35	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-05-13 14:32:35	Added ExtraTreesClassifier to default predictors
2016-05-13 14:32:35	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-05-13 14:32:35	Added ExtraTreesRegressor to default predictors
2016-05-13 14:32:35	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-05-13 14:32:35	Added GradientBoostingClassifier to default predictors
2016-05-13 14:32:35	Predictor GradientBoostingRegressor is of type <class 'sklearn.

In [2]:
folder = '/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian'
!mkdir $folder

figure_folder = '{}/kmer_counting'.format(folder)
! mkdir -p $figure_folder

mkdir: cannot create directory `/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure2_modalities/bayesian': File exists


In [3]:
study.supplemental.modalities_tidy.groupby(['phenotype', 'modality']).size()

phenotype  modality     
MN         bimodal          1316
           excluded         1175
           included         2995
           uncategorized       2
NPC        bimodal          2325
           excluded         1732
           included         4597
           uncategorized       4
iPSC       bimodal          3438
           excluded         2293
           included         6105
           middle              1
           uncategorized      34
dtype: int64

### Make BED files of each modality within each celltype

Use the other modalities, within that celltype, as background.

  - What properties are unique to events that are bimodal in iPSC, rather than ~0 or ~1?
    - Foreground: Bimodal events in iPSC
    - Background: All other events in iPSC

In [4]:


bed_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian'
! mkdir $bed_folder
import pybedtools

mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/figure2_modalities/bayesian': File exists


In [5]:
import pyhomer

In [6]:
all_events = study.supplemental.modalities_tidy.event_id.unique()

In [7]:
from outrigger.region import Region

exon2s = map(lambda x: x.split('@')[1], all_events)
exon2_regions = map(Region, exon2s)
exon2_bed_table = pd.DataFrame(map(lambda x: [x.chrom, x._start, x._stop, x.name, 1000, x.strand], exon2_regions))
exon2_bed_table[3] = all_events
exon2_bed_table.head()

,0,1,2,3,4,5
0,chr10,101163481,101163631,exon:chr10:101165513-101165617:-@exon:chr10:10...,1000,-
1,chr10,101478103,101478257,exon:chr10:101480744-101480825:-@exon:chr10:10...,1000,-
2,chr10,102116289,102116521,exon:chr10:102114184-102114389:+@exon:chr10:10...,1000,+
3,chr10,102286156,102286311,exon:chr10:102286732-102286831:-@exon:chr10:10...,1000,-
4,chr10,102746847,102746953,exon:chr10:102747070-102747240:-@exon:chr10:10...,1000,-


In [8]:
exon2_bed_table = exon2_bed_table.sort_values(by=3)
exon2_bed_table.head()

,0,1,2,3,4,5
9240,chr10,100189548,100189646,exon:chr10:100190328-100190427:-@exon:chr10:10...,1000,-
9241,chr10,100195029,100195171,exon:chr10:100195392-100195529:-@exon:chr10:10...,1000,-
0,chr10,101163481,101163631,exon:chr10:101165513-101165617:-@exon:chr10:10...,1000,-
1,chr10,101478103,101478257,exon:chr10:101480744-101480825:-@exon:chr10:10...,1000,-
5488,chr10,101510126,101510153,exon:chr10:101507014-101507147:+@exon:chr10:10...,1000,+


In [ ]:
exon2_bed_table.to_csv('{}/background_events.bed'.format(bed_folder), index=False, header=False, sep='\t')

In [ ]:
from Bio import SeqIO
import kvector
import pybedtools

DIRECTIONS = 'upstream', 'downstream'



DIR = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'

exon_bedfile = '{}/background_events.bed'.format(bed_folder)
exon_bed = pybedtools.BedTool(exon_bedfile)

commands = []

findMotifsGenome = '/home/yeo-lab/software/homer/bin/findMotifsGenome.pl'
n_processors = 4
homer_flags = '-rna -len 4,5,6 -mset vertebrates -mis 0 -p {} -noweight'.format(n_processors)


primate_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPrimates.bed'
primate = pybedtools.BedTool(primate_filename)
placental_filename = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'
placental = pybedtools.BedTool(placental_filename)
conserved_regions = {'primate': primate, 'placental': placental}

conservation_bed = placental

genome = 'hg19'

nt = 200


genome_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

kmer_zscores = []

for phenotype, phenotype_df in study.supplemental.modalities_tidy.groupby('phenotype'):
    background_events = set(phenotype_df.event_id)
    for modality, modality_df in phenotype_df.groupby('modality'):
        print '---\n', phenotype, modality
        event_ids = set(modality_df.event_id)
        format_args = bed_folder, phenotype, modality
        foreground_table = exon2_bed_table.loc[exon2_bed_table[3].isin(event_ids)]
        foreground_filename = '{}/exon2_{}_{}_foreground.bed'.format(*format_args)
        foreground_table.to_csv(foreground_filename, index=False, header=False, sep='\t')
        foreground = pybedtools.BedTool(foreground_filename)
        
        
        background_table = exon2_bed_table.loc[exon2_bed_table[3].isin(background_events)]
        background_filename = '{}/exon2_{}_{}_background.bed'.format(*format_args)
        background_table.to_csv(background_filename, index=False, header=False, sep='\t')
        background = pybedtools.BedTool(background_filename)
        
        
        pair = pyhomer.ForegroundBackgroundPair(foreground, background)
#         print '\n', pair 
        
        for direction in DIRECTIONS:
            print '\n\t', direction
            intron_pair = pair.flanking_intron(direction, 'hg19', nt)

            conserved_introns = intron_pair.intersect(conservation_bed, 'placental')
#             print '\n', conserved_introns
            %time seqs = conserved_introns.foreground.sequence(fi=genome_fasta, s=True)
            %time foreground_kmers = kvector.count_kmers(seqs.seqfn)

            %time seqs = conserved_introns.background.sequence(fi=genome_fasta, s=True)
            %time background_kmers = kvector.count_kmers(seqs.seqfn)
            kmer_zscore = (foreground_kmers.mean() - background_kmers.mean())/background_kmers.std()
            kmer_zscore.name = '{}{}nt_{}_{}_placental'.format(direction, nt, phenotype, modality)
            kmer_zscores.append(kmer_zscore)
kmer_zscores_all = pd.concat(kmer_zscores, axis=1)
kmer_zscores_all.head()

---
MN bimodal

	upstream
CPU times: user 1e+03 µs, sys: 18 ms, total: 19 ms

In [ ]:
kmer_zscores_all = pd.concat(kmer_zscores, axis=1)
print kmer_zscores_all.shape
kmer_zscores_all = kmer_zscores_all.dropna(axis=1, how='all').dropna(how='all', axis=0)
print kmer_zscores_all.shape

# Replace remaining NAs with zero since they aren't enriched
kmer_zscores_all = kmer_zscores_all.fillna(0)
kmer_zscores_all.head()

In [ ]:
study.supplemental.kmer_zscores = kmer_zscores_all

In [ ]:
range(2)

In [ ]:
kmer_zscores_metadata = pd.DataFrame.from_records(
    study.supplemental.kmer_zscores.columns.map(lambda x: pd.Series(x.split('_'))), 
    index=study.supplemental.kmer_zscores.columns)
kmer_zscores_metadata.columns = ['direction', 'phenotype', 'modality', 'clade']
study.supplemental.kmer_zscores_metadata = kmer_zscores_metadata

kmer_zscores_metadata.head()

In [ ]:
study.supplemental.

In [ ]:
study.save('singlecell_pnm_figure2_modalities_bayesian_kmers', flotilla_dir=flotilla_dir)